Class Life Actions

In [2]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timezone, timedelta
import time

In [4]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        self.window = window
    
    def get_most_recent(self, days=1):
        while True:
            try:
                time.sleep(2)
                now = datetime.now(timezone.utc)
                now = now.replace(microsecond=0)
                past = now - timedelta(days=days)
                
                start = past.strftime('%Y-%m-%dT%H:%M:%SZ')
                end = now.strftime('%Y-%m-%dT%H:%M:%SZ')
                
                print(f"Requesting data from {start} to {end}")
                
                response = self.ctx.instrument.candles(
                    instrument=self.instrument,
                    fromTime=start,
                    toTime=end,
                    granularity="M1",
                    price="M"
                )
                
                # Extract candles from response
                candles = response.body['candles']
                
                # Check if candles are available
                if not candles:
                    print("No candles received from OANDA. Retrying...")
                    time.sleep(5)
                    continue
                
                # Process the candle data using attributes
                data = []
                for candle in candles:
                    if candle.complete:  # Access the 'complete' attribute
                        data.append({
                            'time': pd.to_datetime(candle.time),  # Access the 'time' attribute
                            self.instrument: float(candle.mid.c)  # Access 'mid' price via 'mid.c'
                        })
                
                if not data:
                    print("No complete candles found. Retrying...")
                    time.sleep(5)
                    continue
                
                df = pd.DataFrame(data)
                df.set_index('time', inplace=True)
                
                df = df.resample(self.bar_length, label="right").last().dropna().iloc[:-1]
                self.raw_data = df.copy()
                
                if len(self.raw_data) == 0:
                    print("No data after resampling. Retrying...")
                    time.sleep(5)
                    continue
                
                self.last_bar = self.raw_data.index[-1]
                print(f"Successfully retrieved {len(self.raw_data)} bars of data")
                
                if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length:
                    break
                    
            except Exception as e:
                print(f"Error in get_most_recent: {str(e)}")
                print("Retrying in 5 seconds...")

    def on_success(self, time, bid, ask):
        try:
            print(self.ticks, end=" ")
            recent_tick = pd.to_datetime(time)
            df = pd.DataFrame({self.instrument: (ask + bid) / 2}, index=[recent_tick])
            self.tick_data = pd.concat([self.tick_data, df])

            if recent_tick - self.last_bar > self.bar_length:
                self.resample_and_join()
                self.define_strategy()
                self.execute_trades()
        except Exception as e:
            print(f"Error in on_success: {str(e)}")

    def resample_and_join(self):
        self.raw_data = pd.concat([self.raw_data, self.tick_data.resample(self.bar_length,
                                                                    label="right").last().ffill().iloc[:-1]])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]

    def define_strategy(self):
        df = self.raw_data.copy()
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        self.data = df.copy()

    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress=True, ret=True)
                self.report_trade(order, "GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress=True, ret=True)
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1:
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress=True, ret=True)
                self.report_trade(order, "GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress=True, ret=True)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0:
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress=True, ret=True)
                self.report_trade(order, "GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress=True, ret=True)
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0

    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100 * "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")


Testing connection to Oanda

In [7]:
trader = ConTrader("oanda.cfg", "EUR_USD", bar_length="1min", window=1, units=10000)
account = trader.ctx.account.get(trader.account_id)
print(f"Account ID: {trader.account_id}")

Account ID: 101-001-29655670-001


In [9]:
# Test with just 1 hour of data
now = datetime.now(timezone.utc)
past = now - timedelta(hours=1)
response = trader.ctx.instrument.candles(
    instrument="EUR_USD",
    fromTime=past.strftime('%Y-%m-%dT%H:%M:%SZ'),
    toTime=now.strftime('%Y-%m-%dT%H:%M:%SZ'),
    granularity="M1",
    price="M"
)
print(response.body)

{'instrument': 'EUR_USD', 'granularity': 'M1', 'candles': [<v20.instrument.Candlestick object at 0x000001D571C39820>, <v20.instrument.Candlestick object at 0x000001D571C39880>, <v20.instrument.Candlestick object at 0x000001D571C398E0>, <v20.instrument.Candlestick object at 0x000001D571C39940>, <v20.instrument.Candlestick object at 0x000001D571C399A0>, <v20.instrument.Candlestick object at 0x000001D571C39A00>, <v20.instrument.Candlestick object at 0x000001D571C39A60>, <v20.instrument.Candlestick object at 0x000001D571C39AC0>, <v20.instrument.Candlestick object at 0x000001D571C39B20>, <v20.instrument.Candlestick object at 0x000001D571C39B80>, <v20.instrument.Candlestick object at 0x000001D571C39BE0>, <v20.instrument.Candlestick object at 0x000001D571C39C40>, <v20.instrument.Candlestick object at 0x000001D571C39CA0>, <v20.instrument.Candlestick object at 0x000001D571C39D00>, <v20.instrument.Candlestick object at 0x000001D571C39D60>, <v20.instrument.Candlestick object at 0x000001D571C39DC0

Class exacution

In [ ]:
try:
    # Create trader instance
    trader = ConTrader("oanda.cfg", "EUR_USD", bar_length="1min", window=1, units=10000)
    
    print("Getting most recent data...")
    trader.get_most_recent()  # Fetch the initial data before streaming
    
    print("Starting data stream...")
    trader.stream_data(trader.instrument, stop=200)  # Start streaming data, perform trades
    
    # Check if there is an open position after the stream stops
    if trader.position != 0:
        print(f"Closing open position of {trader.position} units...")

        # Close any remaining open position
        close_order = trader.create_order(
            trader.instrument,
            units=-trader.position * trader.units,  # Opposite trade to close position
            suppress=True,
            ret=True
        )
        
        # Print trade report for the closing order
        trader.report_trade(close_order, "GOING NEUTRAL")
        trader.position = 0
        
except Exception as e:
    print(f"Error in main execution: {str(e)}")


Getting most recent data...
Requesting data from 2024-10-03T00:49:42Z to 2024-10-04T00:49:42Z
Successfully retrieved 1433 bars of data
Requesting data from 2024-10-03T00:49:44Z to 2024-10-04T00:49:44Z
Successfully retrieved 1433 bars of data
Requesting data from 2024-10-03T00:49:47Z to 2024-10-04T00:49:47Z
Successfully retrieved 1433 bars of data
Requesting data from 2024-10-03T00:49:50Z to 2024-10-04T00:49:50Z
Successfully retrieved 1433 bars of data
Requesting data from 2024-10-03T00:49:53Z to 2024-10-04T00:49:53Z
Successfully retrieved 1433 bars of data
Requesting data from 2024-10-03T00:49:55Z to 2024-10-04T00:49:55Z
Successfully retrieved 1433 bars of data
Requesting data from 2024-10-03T00:49:58Z to 2024-10-04T00:49:58Z
Successfully retrieved 1433 bars of data
Requesting data from 2024-10-03T00:50:01Z to 2024-10-04T00:50:01Z
Successfully retrieved 1433 bars of data
Requesting data from 2024-10-03T00:50:03Z to 2024-10-04T00:50:03Z
Successfully retrieved 1433 bars of data
Requestin